In [1]:
from pyspark.sql import SparkSession
from dotenv import load_dotenv
import os

spark = SparkSession.builder \
    .appName("Localspark") \
    .master("local[*]") \
    .config("spark.jars.packages",
        "com.microsoft.sqlserver:mssql-jdbc:12.6.1.jre11") \
    .getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

:: loading settings :: url = jar:file:/opt/conda/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
com.microsoft.sqlserver#mssql-jdbc added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-2d389a7f-aea2-4b32-8667-a57d2bb60ef2;1.0
	confs: [default]
	found com.microsoft.sqlserver#mssql-jdbc;12.6.1.jre11 in central
downloading https://repo1.maven.org/maven2/com/microsoft/sqlserver/mssql-jdbc/12.6.1.jre11/mssql-jdbc-12.6.1.jre11.jar ...
	[SUCCESSFUL ] com.microsoft.sqlserver#mssql-jdbc;12.6.1.jre11!mssql-jdbc.jar (1245ms)
:: resolution report :: resolve 1554ms :: artifacts dl 1250ms
	:: modules in use:
	com.microsoft.sqlserver#mssql-jdbc;12.6.1.jre11 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	-----------------------------------------------------------

In [2]:
# Load variables from .env into os.environ
load_dotenv("env")
server_name = os.environ.get("SERVERNAME")
database_name = os.environ.get("DATABASENAME")
username = os.environ.get("USERNAME")
password = os.environ.get("PASSWORD")

In [9]:
password

'qayWSXedc123$%'

In [10]:
# Standard JDBC URL format for SQL Server
jdbc_url = f"jdbc:sqlserver://{server_name}:1433;databaseName={database_name}"

#table name to read
table_name = "dbo.CUSTOMER"

In [11]:
try:
    jdbcDF = spark.read \
        .format("jdbc") \
        .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
        .option("url", jdbc_url) \
        .option("dbtable", table_name) \
        .option("user", username) \
        .option("password", password) \
        .load()

    # Display schema and a few rows if successful
    jdbcDF.printSchema()
    jdbcDF.show(5)

except Exception as e:
    print("JDBC read failed!")
    print("Error:", e)

root
 |-- CUST_ID: string (nullable = true)
 |-- NAME: string (nullable = true)
 |-- ADDRESS: string (nullable = true)
 |-- CITY: string (nullable = true)
 |-- STATE: string (nullable = true)
 |-- POSTALCODE: integer (nullable = true)
 |-- COUNTRY: string (nullable = true)
 |-- PHONE: string (nullable = true)
 |-- LAST_ORDER_DATE: timestamp (nullable = true)
 |-- STATUS: string (nullable = true)
 |-- TIER: string (nullable = true)
 |-- MODIFIED_DATE: timestamp (nullable = true)



+-------+---------------+--------------------+-----------+-----+----------+-------+--------------+-------------------+------+----+-------------------+
|CUST_ID|           NAME|             ADDRESS|       CITY|STATE|POSTALCODE|COUNTRY|         PHONE|    LAST_ORDER_DATE|STATUS|TIER|      MODIFIED_DATE|
+-------+---------------+--------------------+-----------+-----+----------+-------+--------------+-------------------+------+----+-------------------+
|   1096|   Paul Somogye|3130 Broadway Street|Kansas City|   MO|     64111|     US|(816) 756-1060|2009-06-08 00:00:00|  LIVE|   3|2019-06-24 00:00:00|
|   1099|    Peter Doyle|1200 New Hampshir...| Washington|   DC|     20036|     US|(202) 857-2634|2009-02-03 00:00:00|  LIVE|NULL|2019-06-25 00:00:00|
|   1100|   Peter Howley|  25 Cornwall Street|Mill Valley|   CA|     94941|   NULL|(415) 394-5800|2008-09-04 00:00:00|  LIVE|NULL|2019-06-28 00:00:00|
|   1101|  Rick Sundheim|  90 Inverness Cir E|  Englewood|   CO|     80112|     US|    3037990